In [69]:
# Import packages
import pandas as pd
import numpy as np

# Load dataset
airlines = pd.read_csv('airlines.csv')
airports = pd.read_csv('airports.csv')
delay = pd.read_csv('delay.csv')
flights_details = pd.read_csv('flights_details.csv', parse_dates=["departure_time", "arrival_time"])
flights_info = pd.read_csv('flights_info.csv')
planes = pd.read_csv('planes.csv', sep=";")


In [70]:
#EDA
dataframes= {'airlines': airlines, 'airports': airports, 'delay': delay, 
            'flights_details': flights_details, 'flights_info': flights_info, 'planes': planes}

for name, df in dataframes.items():
    print(f"DataFrame: {name}")
    print("--------------")
    print(df.head())
    print(df.info())
    print("\n")

DataFrame: airlines
--------------
  airline_id                                  airline_name         country
0        GNL                                   135 Airways   United States
1        RNX                                 1Time Airline    South Africa
2        WYT  2 Sqn No 1 Elementary Flying Training School  United Kingdom
3        TFU                               213 Flight Unit          Russia
4        CHD                 223 Flight Unit State Airline          Russia
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5808 entries, 0 to 5807
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   airline_id    5808 non-null   object
 1   airline_name  5808 non-null   object
 2   country       5803 non-null   object
dtypes: object(3)
memory usage: 136.3+ KB
None


DataFrame: airports
--------------
  airport_id                                 airport_name          city  \
0       AYGA                            

In [71]:
# Convert data
#delay["delay"]=pd.to_timedelta(delay["delay"], unit="minute")

flights_details = flights_details.astype({'status': 'category'})



### 1. Combien y a-t-il de vols en cours et combien de destinations différentes sont desservies ?

In [72]:
# Fight ongoing
flights_details.pivot_table(index="status", values="flight_id", aggfunc='count')


C:\Users\J-P-F\AppData\Local\Temp\ipykernel_67124\3659173356.py:2: FutureWarning: The default value of observed=False is deprecated and will change to observed=True in a future version of pandas. Specify observed=False to silence this warning and retain the current behavior
  flights_details.pivot_table(index="status", values="flight_id", aggfunc='count')


,flight_id
status,
Delayed,282
Estimated,980
Landed,111
Scheduled,27
Unknown,4


In [73]:
# Number of destination
len(airports.city)

7698

### 2. Quelle est la compagnie aérienne avec le plus grand nombre de vols pour une journée donnée ?

In [ ]:
# Merg dataset
flights=flights_details.merge(flights_info, on="flight_id")
flights_airlines=flights.merge(airlines, on="airline_id")
flights_airlines.head()

# Create a departure date column
flights_airlines["departure_date"]=flights_airlines["departure_time"].dt.date

# Pivot table with date in the index
number_derpature=flights_airlines.pivot_table(index="departure_date", columns="airline_name", 
                             values="flight_id", aggfunc="count", fill_value=0, sort=True)

# Number max of flight during a day
number_derpature.values.max()

60

In [ ]:
# Day with the highest number of derpature for a company
print(number_derpature.max(1))
print("\n")
# Comapny with the highest number of derpature
print(number_derpature.idxmax(axis=1))
print("\n")
# Checking
print(number_derpature.loc[:,"United Airlines"])

departure_date
1970-01-01     2
2024-12-15     7
2024-12-16    60
dtype: int64


departure_date
1970-01-01            DHL Air
2024-12-15      Qatar Airways
2024-12-16    United Airlines
dtype: object


departure_date
1970-01-01     0
2024-12-15     0
2024-12-16    60
Name: United Airlines, dtype: int64


### 3. Quels sont les trois avions les plus utilisés (modèle d'avion) et combien d'heures de vol ont-ils effectuées ?

In [92]:
# Marge datadet
flights_planes=flights_airlines.merge(planes, on="plane_id")
flights_planes.head()


,flight_id,departure_time,arrival_time,passenger_count,status,flight_number,origin_airport_id,destination_airport_id,airline_id,plane_id,airline_name,country,departure_date,manufacturer,model,seating_capacity
0,385baa85,2024-12-15 14:42:00+00:00,2024-12-15 21:29:00+00:00,189,Landed,FX9887,KMEM,PANC,FDX,B77L,Federal Express,United States,2024-12-15,BOEING,Boeing 777-200LRF/LR,191
1,385c1e68,1970-01-01 00:00:00+00:00,1970-01-01 00:00:00+00:00,191,Estimated,ET506,HAAB,SBGR,ETH,B77L,Ethiopian Airlines,Ethiopia,1970-01-01,BOEING,Boeing 777-200LRF/LR,191
2,385c3e91,2024-12-15 19:55:00+00:00,2024-12-16 12:55:00+00:00,116,Landed,DL201,FAOR,KATL,DAL,A359,Delta Air Lines,United States,2024-12-15,AIRBUS,Airbus 350-900,149
3,385c6eee,2024-12-15 22:15:00+00:00,2024-12-16 15:55:00+00:00,2,Estimated,QF4,KJFK,NZAA,QFA,B789,Qantas,Australia,2024-12-15,BOEING,Boeing 787-9 Dreamliner,2
4,385c905b,2024-12-15 23:00:00+00:00,2024-12-16 14:55:00+00:00,127,Estimated,QR920,OTHH,NZAA,QTR,A35K,Qatar Airways,Qatar,2024-12-15,AIRBUS,Airbus A350-1000 XWB,178


In [ ]:
# The 3 most used flights
flights_planes.pivot_table(index="model", values="flight_id", aggfunc="count").nlargest(3, "flight_id")

,flight_id
model,
Boeing 777-300ER,206
Boeing 787-9 Dreamliner,192
Airbus 350-900,115


In [ ]:
# Flifhts duration
flights_planes["flights_duration"]= flights_planes["arrival_time"]-flights_planes["departure_time"]

flights_planes.loc[flights_planes.model.isin(["Boeing 777-300ER", 
                                              "Boeing 787-9 Dreamliner", 
                                              "Airbus 350-900"])].pivot_table(index="model",
                                                                              values="flights_duration",
                                                                              aggfunc="sum").sort_values("flights_duration", ascending=False)

,flights_duration
model,
Boeing 777-300ER,99 days 14:20:00
Boeing 787-9 Dreamliner,91 days 04:59:00
Airbus 350-900,57 days 10:20:00
